In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [4]:
credentials_location = '/home/derekwong/.gc/de-project-service_account-credential.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [5]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/04/04 17:20:40 WARN Utils: Your hostname, DavidNg-8242-NB resolves to a loopback address: 127.0.1.1; using 172.24.225.125 instead (on interface eth0)
23/04/04 17:20:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/04 17:20:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [12]:
df_202203 = spark.read.parquet('gs://dtc_data_lake_de-project-381616/pq/china/hk/hong-kong/2022/03/*')

In [11]:
df_202206 = spark.read.parquet('gs://dtc_data_lake_de-project-381616/pq/china/hk/hong-kong/2022/06/*')

In [23]:
df_202209 = spark.read.parquet('gs://dtc_data_lake_de-project-381616/pq/china/hk/hong-kong/2022/09/*')

In [24]:
df_202212 = spark.read.parquet('gs://dtc_data_lake_de-project-381616/pq/china/hk/hong-kong/2022/12/*')

In [13]:
import pyspark.sql.functions as F

In [14]:
df_202203 = df_202203 \
    .withColumn('source_date', F.lit('202203'))

In [25]:
df_202206 = df_202206 \
    .withColumn('source_date', F.lit('202206'))

In [26]:
df_202209 = df_202209 \
    .withColumn('source_date', F.lit('202209'))

In [27]:
df_202212 = df_202212 \
    .withColumn('source_date', F.lit('202212'))

In [20]:
df_202206.columns

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availa

In [37]:
df = df_202203.unionByName(df_202206, allowMissingColumns=True)

In [40]:
df = df.unionByName(df_202209, allowMissingColumns=True)

In [42]:
df = df.unionByName(df_202212, allowMissingColumns=True)

In [43]:
df.count()

23963

In [44]:
df = df.withColumn('price_in_numbers', F.regexp_replace('price', '[$,]', '').cast('double'))

In [45]:
df.createOrReplaceTempView('hk_data')

In [46]:
spark.sql("""
SELECT
source_date,
count(1)
FROM
    hk_data
Group by 
source_date
""").show()

+-----------+--------+
|source_date|count(1)|
+-----------+--------+
|     202203|    6092|
|     202206|    7620|
|     202209|    5057|
|     202212|    5194|
+-----------+--------+



In [ ]:
spark.sql("""
SELECT
price,
price_in_numbers
FROM
    hk_data
WHERE
    availability_30>0
""").show()

In [47]:
spark.sql("""
SELECT
neighbourhood_cleansed,
room_type,
avg(availability_30),
avg(availability_60),
avg(accommodates),
avg(price)
FROM
    hk_data
WHERE
    availability_30>0
group by neighbourhood_cleansed,room_type
""").show()

+----------------------+---------------+--------------------+--------------------+------------------+----------+
|neighbourhood_cleansed|      room_type|avg(availability_30)|avg(availability_60)| avg(accommodates)|avg(price)|
+----------------------+---------------+--------------------+--------------------+------------------+----------+
|          Sham Shui Po|     Hotel room|                23.0|               52.25|               2.0|      null|
|          Wong Tai Sin|    Shared room|                29.0|                59.0|               1.0|      null|
|          Wong Tai Sin|Entire home/apt|  16.333333333333332|   42.44444444444444| 3.888888888888889|      null|
|          Sham Shui Po|    Shared room|   28.38888888888889|   57.55555555555556|1.4722222222222223|      null|
|     Central & Western|Entire home/apt|  21.411246612466126|  46.076558265582655|2.4905149051490514|      null|
|              Wan Chai|    Shared room|   28.88135593220339|   58.66101694915254|              